# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('api_key')

In [3]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [35]:
# Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ["authors","publsihers"],  # ENTER A TABLE COLUMNS HERE
        'definition': ["authors names","puiblisher"]}  # ENTER A TABLE DEFINITIONS HERE

# Load the CSV file into a DataFrame
authors_df = pd.read_csv("authors.csv",)
publisher_df = pd.read_csv("publishers.csv",)
print(authors_df,publisher_df)

          au_id        au_lname     au_fname         phone  \
0   172-32-1176           White      Johnson  408 496-7223   
1   213-46-8915           Green     Marjorie  415 986-7020   
2   238-95-7766          Carson       Cheryl  415 548-7723   
3   267-41-2394         O'Leary      Michael  408 286-2428   
4   274-80-9391        Straight         Dean  415 834-2919   
5   341-22-1782           Smith      Meander  913 843-0462   
6   409-56-7008          Bennet      Abraham  415 658-9932   
7   427-17-2319            Dull          Ann  415 836-7128   
8   472-27-2349      Gringlesby         Burt  707 938-6445   
9   486-29-1786        Locksley     Charlene  415 585-4620   
10  527-72-3246          Greene  Morningstar  615 297-2723   
11  648-92-1872  Blotchet-Halls     Reginald  503 745-6402   
12  672-71-3249        Yokomoto        Akiko  415 935-4228   
13  712-45-1867    del Castillo        Innes  615 996-8275   
14  722-51-5454        DeFrance       Michel  219 547-9982   
15  724-

In [50]:
# Combine information from authors_df
authors_text = '\n'.join([
    f"{row['au_id']}, {row['au_lname']}, {row['au_fname']}" 
    for index, row in authors_df.iterrows()
])

# Combine information from publishers_df
publishers_text = '\n'.join([
    f"{row['pub_id']}, {row['pub_name']}, {row['city']}" 
    for index, row in publisher_df.iterrows()
])

# Combine both into text_tables
text_tables = f"Authors:\n{authors_text}\n\nPublishers:\n{publishers_text}"

In [51]:
print(text_tables)

Authors:
172-32-1176, White, Johnson
213-46-8915, Green, Marjorie
238-95-7766, Carson, Cheryl
267-41-2394, O'Leary, Michael
274-80-9391, Straight, Dean
341-22-1782, Smith, Meander
409-56-7008, Bennet, Abraham
427-17-2319, Dull, Ann
472-27-2349, Gringlesby, Burt
486-29-1786, Locksley, Charlene
527-72-3246, Greene, Morningstar
648-92-1872, Blotchet-Halls, Reginald
672-71-3249, Yokomoto, Akiko
712-45-1867, del Castillo, Innes
722-51-5454, DeFrance, Michel
724-08-9931, Stringer, Dirk
724-80-9391, MacFeather, Stearns
756-30-7391, Karsen, Livia
807-91-6654, Panteley, Sylvia
846-92-7186, Hunter, Sheryl
893-72-1158, McBadden, Heather
899-46-2035, Ringer, Anne
998-72-3567, Ringer, Albert

Publishers:
736, New Moon Books, Boston
877, Binnet & Hardley, Washington
1389, Algodata Infosystems, Berkeley
1622, Five Lakes Publishing, Chicago
1756, Ramona Publishers, Dallas
9901, GGG&G, TcKBbmNoZW4=
9952, Scootney Books, New York
9999, Lucerne Publishing, Paris


In [52]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
authors: Contains information about authors, such as name, address, and phone number.
publishers: Contains information about publishers, such as name, city, and country.

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
Which authors live in California?
"""


In [53]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="What phone number does Green have?")#ENTER YOUR QUERY HERE)

In [54]:
print(return_OAI(pqt1))

{
    "tables": ["authors"]
}


In [46]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="whast the id of green?")#ENTER YOUR QUERY HERE)

In [55]:
print(return_OAI(pqt3))

{
    "tables": ["authors"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

i learned that GPT-3.5-Turbo can effectively select relevant tables based on their names and brief descriptions, but it may struggle with complex queries or when the context is not clear enough. Variations in prompts can lead to different levels of success, with more detailed descriptions generally yielding better results.